In [14]:
from pathlib import Path
import requests
from PIL import Image

In [15]:
def read_image_as_bytes(image_path, fail_silently=True):
    if not Path(image_path).exists(): 
        if fail_silently: return None
        else: raise ValueError(f"{image_path} does not exist")
    
    with open(image_path, "rb") as image:
        b = bytearray(image.read())

    return b

In [16]:
v = read_image_as_bytes('sample DL rotated.jpeg')
print(type(v), len(v))
# from PIL import Image
# a = Image.open('sample DL.jpeg')
# print(type(a), a.width, a.height)
# a.show()
# b = a.rotate(90, Image.NEAREST, expand=1)
# b.show()
# b.save('sample DL rotated.jpeg')

<class 'bytearray'> 80261


In [17]:
resp = requests.post("http://103.93.17.125:5001/api/v2/docs",
    files={"file": v, "file1": v},
    data={"document_type": "LICENCE", "country": 'IND', "state": 'MH',\
    "user": 'test_user_ocr', "code": '0000111100001111'}
)

In [18]:
# resp.json()
resp

<Response [200]>

In [49]:
import shutil
import numpy as np
resp1 = requests.get('http://103.93.17.125:5001/static/img/ims/irof4aThwCY8IXH7.jpg', stream=True)
with open('saved.png', 'wb') as f:
    resp1.raw.decode_content = True
    shutil.copyfileobj(resp1.raw, f)

In [30]:
resp.json()

{'face_url': 'http://103.93.17.125:5001/static/img/ims/BqdT7EP9dpcizVON.jpg',
 'field_values': [{'value': 'MH02 20200011373'},
  {'value': 'AUTHORISA TION TO DRI'},
  {'value': '10-09-1999'},
  {'value': 'AUTHORISA TION TO DRI VE FOLLOWING CLASS'},
  {'value': 'A'},
  {'value': ' '},
  {'value': '31-08-2020'},
  {'value': '31-08-2020'},
  {'value': '-1'}],
 'fields_detected': [{'value': 'LICENCE NO'},
  {'value': 'NAME'},
  {'value': 'DOB'},
  {'value': 'CARE OF'},
  {'value': 'CLASS'},
  {'value': 'ADDRESS'},
  {'value': 'DATE OF ISSUE'},
  {'value': 'DATE OF EXPIRY'},
  {'value': 'ID'}]}

In [1]:
import face_recognition
from numpy import frombuffer, ndarray, array
from pathlib import Path

class FaceRecognition:

    class ImageDoesNotExists(Exception): pass
    class NoFaceDetected(Exception): pass
    @staticmethod
    def get_face_encodings(image_path):
        if not Path(image_path).exists(): raise FaceRecognition.ImageDoesNotExists('check image path')
        image_loaded = face_recognition.load_image_file(image_path)
        image_encodings = face_recognition.face_encodings(image_loaded)

        if not image_encodings: raise FaceRecognition.NoFaceDetected('No face was detected in selfie')    
        image_encodings = image_encodings[0]
        # print('-->', image_encodings[1:])

        return image_encodings.tobytes()

    class InvalidStringEncoding(Exception): pass
    @staticmethod
    def get_encodings_from_str(encodings) -> ndarray:
        if not isinstance(encodings, bytes): raise FaceRecognition.InvalidStringEncoding('encodings should be of type bytes')

        return frombuffer(encodings, dtype='float64')


    class InvalidEncoding(Exception): pass
    @staticmethod
    def compare_encodings(encoding1, encoding2):
        if not isinstance(encoding1, ndarray) and not isinstance(encoding2, ndarray):
            raise FaceRecognition.InvalidEncoding('encoding provided not ndarray')
        return face_recognition.compare_faces([encoding1], encoding2)

    @staticmethod
    def compare_distance(encoding1, encoding2):
        if not isinstance(encoding1, ndarray) and not isinstance(encoding2, ndarray):
            raise FaceRecognition.InvalidEncoding('encoding provided not ndarray')

        return face_recognition.face_distance([encoding1], encoding2)

In [12]:
doc_pp = FaceRecognition.get_face_encodings('saved.jpg')
doc_pp = FaceRecognition.get_encodings_from_str(doc_pp)
selfie = FaceRecognition.get_face_encodings('test user 2_backside.png')
selfie = FaceRecognition.get_encodings_from_str(selfie)
print(type(doc_pp), type(selfie))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [10]:
r1 = FaceRecognition.compare_encodings(selfie, doc_pp)
r2 = FaceRecognition.compare_distance(selfie, doc_pp)

In [11]:
print(r1, r2)

[True] [0.43695314]


In [26]:
rj = resp.json()

formated = {}
for fields, values in zip(rj['fields_detected'], rj['field_values']):
    formated[fields['value']] = values['value']

formated['face_url'] = rj['face_url']
rj, formated

({'face_url': 'http://103.93.17.125:5001/static/img/ims/lC7pUVhZXP5IeviR.jpg',
  'field_values': [{'value': 'MH02 20200011373'},
   {'value': 'AUTHORISA TION TO DRI'},
   {'value': '10-09-1999'},
   {'value': 'AUTHORISA TION TO DRI VE FOLLOWING CLASS'},
   {'value': 'A'},
   {'value': ' '},
   {'value': '31-08-2020'},
   {'value': '31-08-2020'},
   {'value': '-1'}],
  'fields_detected': [{'value': 'LICENCE NO'},
   {'value': 'NAME'},
   {'value': 'DOB'},
   {'value': 'CARE OF'},
   {'value': 'CLASS'},
   {'value': 'ADDRESS'},
   {'value': 'DATE OF ISSUE'},
   {'value': 'DATE OF EXPIRY'},
   {'value': 'ID'}]},
 {'LICENCE NO': 'MH02 20200011373',
  'NAME': 'AUTHORISA TION TO DRI',
  'DOB': '10-09-1999',
  'CARE OF': 'AUTHORISA TION TO DRI VE FOLLOWING CLASS',
  'CLASS': 'A',
  'ADDRESS': ' ',
  'DATE OF ISSUE': '31-08-2020',
  'DATE OF EXPIRY': '31-08-2020',
  'ID': '-1',
  'face_url': 'http://103.93.17.125:5001/static/img/ims/lC7pUVhZXP5IeviR.jpg'})

In [50]:
a = ['INITIALIZED', 'SELFIE', 'DOCTYPE', 'FRONTSIDE', 'BACKSIDE', 'RESULT', 'FINISHED']

b = {}
p = None
for v in a:
    if not p:
        p = v
        continue

    b[p] = v
    p = v
b

{'INITIALIZED': 'SELFIE',
 'SELFIE': 'DOCTYPE',
 'DOCTYPE': 'FRONTSIDE',
 'FRONTSIDE': 'BACKSIDE',
 'BACKSIDE': 'RESULT',
 'RESULT': 'FINISHED'}